In [7]:
import numpy as np
import pandas as pd
import gzip
import json
import matplotlib.pyplot as plt
import torch

import os
from os import walk
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

2022-02-04 15:55:54.449241: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-04 15:55:54.449312: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


KeyboardInterrupt: 

In [3]:
#get path informationb
product_path = '../../../../src/data/product'
train_test_all_filtered_path = os.path.join(product_path, 'train_test_split/output_unfiltered_tables/large/after_manual_checking/baselines')


In [4]:
df_train = pd.read_csv(os.path.join(train_test_all_filtered_path,'df_train'))
df_val = pd.read_csv(os.path.join(train_test_all_filtered_path,'df_val'))
df_test = pd.read_csv(os.path.join(train_test_all_filtered_path,'df_test'))

/home/bizer-tp2021/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (33,36,43,48,49,50,53,59,61,62,64,65,66,67,68,69) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/bizer-tp2021/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (33,35,36,37,49,53,54,55,57,61,70,73,75,77,78,80,81,82) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
target_train = df_train['cluster_id_mapped']
target_val = df_val['cluster_id_mapped']
target_test = df_test['cluster_id_mapped']

In [ ]:
x_train = df_train['concat'].to_list()
x_test = df_test['concat'].to_list()
x_val = df_val['concat'].to_list()
y_val = target_val.to_list()
y_train = target_train.to_list()
y_test = target_test.to_list()

In [10]:
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast

tokenizer = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-bert-for-token-classification", max_length = 512)
model = AutoModelForTokenClassification.from_pretrained("hf-internal-testing/tiny-bert-for-token-classification", num_labels=target_train.nunique())

2021-12-01 21:59:25.396637: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-01 21:59:25.396737: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are ini

In [20]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = r'./TinyBert_Results',
    num_train_epochs=25,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,
    report_to='none',
    logging_dir=r'./TinyBert_logs',
    evaluation_strategy="epoch",
)

PyTorch: setting up devices


In [13]:
class dataDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [15]:
x_Train = tokenizer(x_train, truncation=True, padding='max_length')
x_Test = tokenizer(x_test, truncation=True, padding='max_length')
x_Val = tokenizer(x_val, truncation=True, padding='max_length')


train_data = dataDataset(x_Train, y_train)
test_data = dataDataset(x_Test, y_test)
val_data = dataDataset(x_Val, y_val)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [22]:
trainer.train()

***** Running training *****
  Num examples = 19466
  Num Epochs = 25
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 121675


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate(eval_dataset=test_data)